In [1]:
# set up the config
class Config:
    BATCH_SIZE = 16
    MAX_LEN = 128
    TARGET = 'label'
    PROJECT = 'wsd'
    MODEL = 'xlm-roberta-base'
    LEARNING_RATE = 1e-05
    EPOCHS = 10 # 5
    EPS = 1e-08
    random_seed = 0xfeedbeef
    dataset = "EENLP.WSD.WiC"
    full_data = "wsd.wic.english.jsonl"
    eval_data = {
        "bulgarian" :"wsd.wic.bulgarian.jsonl",
        "croatian" :"wsd.wic.croatian.jsonl",
        "estonian" :"wsd.wic.estonian.jsonl",
        "hungarian"  :"wsd.wic.hungarian.jsonl",
        "russian"  :"wsd.wic.russian.jsonl",
        
    }


In [2]:
# get the data from Google Drive public link
!gdown --id 1gk7evveMDbaXj7vw72ZuJnyu08bN5NZC
!unzip `ls /content/*.zip`

"gdown" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"unzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
# Check if we have GPU
!nvidia-smi

In [ ]:
# prepare env

!pip install transformers
!pip install wget
!pip install urllib2
!pip install wandb -qqq
!pip install jsonlines

In [3]:
import json
import random

import torch
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
from transformers import get_linear_schedule_with_warmup, AdamW


In [4]:
# Log in to your W&B account
wandb.login()
wandb.init(
      entity="eenlp",
      project= Config.PROJECT, 
      # Track hyperparameters and run metadata
      config=dict([(k,v) for k,v in Config.__dict__.items() if k[0]!='_']),
      reinit=True
)
#     run = wandb.init(project="storydb_eval.task3", reinit=True)
wandb.run.name += f'_{Config.MODEL}'
wandb.run.save()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

wandb: Currently logged in as: altsoph (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


device(type='cuda')

In [5]:
label_encoder = None
labels_codes = None

def load_dataset(fn):
    return pd.read_json(fn, lines=True) 

def load_dataset_and_split(fn, fraction=.8):
    data = pd.read_json(fn, lines=True)
    df_train=data.sample(frac=fraction,random_state=200)
    df_test=data.drop(df_train.index).reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    return df_train, df_test

def process_dataset(data, tokenizer, seq=False):
    global label_encoder, labels_codes
    if label_encoder is None:
        print('init of label encoder')
        label_encoder = LabelEncoder().fit(data[Config.TARGET])
        keys = list(sorted(set(data[Config.TARGET])))
        labels = label_encoder.transform(keys)
        labels_codes = dict(zip(keys, labels))
    data[Config.TARGET] = label_encoder.transform(data[Config.TARGET])

    input_ids = torch.tensor([])
    attention_masks = torch.tensor([])

    data['TEXT'] = data['sentence1'] + ' [SEP] ' + data['sentence1'] + ' [SEP] ' + data['word']
    for sent in data.loc[:, 'TEXT']:
        encoded_sent = tokenizer.encode_plus(sent, add_special_tokens = True,
                                             max_length = Config.MAX_LEN, 
                                             padding = 'max_length',
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_tensors = 'pt')
        input_ids = torch.cat([input_ids, encoded_sent['input_ids']])
        attention_masks = torch.cat([attention_masks, encoded_sent['attention_mask']])
    labels = torch.tensor(data[Config.TARGET])
    dataset = TensorDataset(input_ids, attention_masks, labels)
    if seq:
        return DataLoader(dataset, sampler = SequentialSampler(dataset), batch_size = Config.BATCH_SIZE)
    else:
        return DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = Config.BATCH_SIZE)


In [6]:
# fix PRNG
random.seed(Config.random_seed)
np.random.seed(Config.random_seed)
torch.manual_seed(Config.random_seed)
torch.cuda.manual_seed_all(Config.random_seed)

# init tokenizer
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL, truncation=True, do_lower_case=False)

# split english dataset
print('parsing and preparing data, it will take a while.')
print('english', end='... \t')
train_df, test_df = load_dataset_and_split(Config.full_data, .8)
train_loader = process_dataset(train_df, tokenizer, seq=False)
eval_loaders = dict()
eval_loaders['english'] = process_dataset(test_df, tokenizer, seq=True)
print('done.')
# prepare eval for other languages
for lang, filename in Config.eval_data.items():
    print(lang, end='... \t')
    eval_loaders[lang] = process_dataset(load_dataset(filename), tokenizer, seq=True)
    print('done.')



parsing and preparing data, it will take a while.
english... 	init of label encoder
done.
bulgarian... 	done.
croatian... 	done.
estonian... 	done.
hungarian... 	done.
russian... 	done.


In [7]:
# Init model
model = AutoModelForSequenceClassification.from_pretrained(
   Config.MODEL,
   num_labels = len(labels_codes),
   output_attentions = False,
   output_hidden_states = False    
)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [8]:
def train(model, epoch, loader, scheduler=None):
    model.train()

    train_loss_accum = 0
    fin_targets=[]
    fin_outputs=[]
    
    for index, (sentence, attention_mask, label) in tqdm(enumerate(loader)):
        model.zero_grad()

        sentence = sentence.to(device).long()
        attention_mask = attention_mask.to(device).long()
        label = label.to(device).long()

        output = model(sentence, attention_mask = attention_mask, labels = label)
        loss_value, logits = output[0], output[1]
        train_loss_accum += loss_value.item()
        fin_targets.extend(label.cpu().detach().numpy().tolist())
        logits = logits.cpu().detach().numpy()
        fin_outputs.extend(np.argmax(logits, axis=1))
        
        loss_value.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

    avg_loss = train_loss_accum / index
    train_accuracy = metrics.accuracy_score( fin_targets, fin_outputs )
#     train_f1_micro = metrics.f1_score(fin_targets, fin_outputs, average='micro')
#     train_f1_macro = metrics.f1_score(fin_targets, fin_outputs, average='macro')

    wandb.log({"train/loss": avg_loss, 
               "train/acc":  train_accuracy,
               "train/f1" : metrics.f1_score(fin_targets, fin_outputs),
               "train/prec" : metrics.precision_score(fin_targets, fin_outputs),
               "train/rec" : metrics.recall_score(fin_targets, fin_outputs),
#                "train/f1_macro" : train_f1_macro,
               "epoch":epoch,
              })



In [9]:
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for sentence, attention_mask, targets in testing_loader:
            sentence = sentence.to(device).long()
            attention_mask = attention_mask.to(device).long()
            outputs = model(sentence, attention_mask = attention_mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            logits = outputs.logits.cpu().detach().numpy()
            fin_outputs.extend(np.argmax(logits, axis=1))
#             break
    return fin_targets, fin_outputs

def eval_model(model, epoch=-1):
    for lang, eval_loader in eval_loaders.items():
        targets, preds = validation(model, eval_loader)
        scores = dict()
        scores[f'valid/acc/{lang}'] = metrics.accuracy_score( targets, preds )
        scores[f'valid/f1/{lang}'] = metrics.f1_score(targets, preds) # , average='micro')
#         scores[f'valid/f1_macro/{lang}'] = metrics.f1_score(targets, preds, average='macro')
        scores[f'valid/prec/{lang}'] = metrics.precision_score(targets, preds)
        scores[f'valid/rec/{lang}'] = metrics.recall_score(targets, preds)

        scores['epoch'] = epoch
        print(scores)
        wandb.log(scores)


In [10]:
optimizer = AdamW(model.parameters(), lr = Config.LEARNING_RATE, eps = Config.EPS)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, 
                                            num_training_steps = Config.EPOCHS*len(train_df)/Config.BATCH_SIZE)

eval_model(model, epoch=-1)
for epoch in range(Config.EPOCHS):
    train(model, epoch, train_loader, scheduler)
    eval_model(model, epoch)


{'valid/acc/english': 0.5012366034624897, 'valid/f1/english': 0.6677649643053266, 'valid/prec/english': 0.5012366034624897, 'valid/rec/english': 1.0, 'epoch': -1}
{'valid/acc/bulgarian': 0.2249774571686204, 'valid/f1/bulgarian': 0.36731689363268316, 'valid/prec/bulgarian': 0.2249774571686204, 'valid/rec/bulgarian': 1.0, 'epoch': -1}
{'valid/acc/croatian': 0.1015625, 'valid/f1/croatian': 0.18439716312056736, 'valid/prec/croatian': 0.1015625, 'valid/rec/croatian': 1.0, 'epoch': -1}
{'valid/acc/estonian': 0.10040983606557377, 'valid/f1/estonian': 0.18249534450651766, 'valid/prec/estonian': 0.10040983606557377, 'valid/rec/estonian': 1.0, 'epoch': -1}
{'valid/acc/hungarian': 0.49512987012987014, 'valid/f1/hungarian': 0.6623235613463627, 'valid/prec/hungarian': 0.49512987012987014, 'valid/rec/hungarian': 1.0, 'epoch': -1}


0it [00:00, ?it/s]

{'valid/acc/russian': 0.37, 'valid/f1/russian': 0.5401459854014599, 'valid/prec/russian': 0.37, 'valid/rec/russian': 1.0, 'epoch': -1}


304it [01:45,  2.87it/s]


{'valid/acc/english': 0.5012366034624897, 'valid/f1/english': 0.6677649643053266, 'valid/prec/english': 0.5012366034624897, 'valid/rec/english': 1.0, 'epoch': 0}
{'valid/acc/bulgarian': 0.2249774571686204, 'valid/f1/bulgarian': 0.36731689363268316, 'valid/prec/bulgarian': 0.2249774571686204, 'valid/rec/bulgarian': 1.0, 'epoch': 0}
{'valid/acc/croatian': 0.1015625, 'valid/f1/croatian': 0.18439716312056736, 'valid/prec/croatian': 0.1015625, 'valid/rec/croatian': 1.0, 'epoch': 0}
{'valid/acc/estonian': 0.10040983606557377, 'valid/f1/estonian': 0.18249534450651766, 'valid/prec/estonian': 0.10040983606557377, 'valid/rec/estonian': 1.0, 'epoch': 0}
{'valid/acc/hungarian': 0.49512987012987014, 'valid/f1/hungarian': 0.6623235613463627, 'valid/prec/hungarian': 0.49512987012987014, 'valid/rec/hungarian': 1.0, 'epoch': 0}


...

{'valid/acc/english': 0.6224237427864798, 'valid/f1/english': 0.6388012618296529, 'valid/prec/english': 0.6136363636363636, 'valid/rec/english': 0.6661184210526315, 'epoch': 8}
{'valid/acc/bulgarian': 0.37060414788097384, 'valid/f1/bulgarian': 0.3608058608058608, 'valid/prec/bulgarian': 0.2338278931750742, 'valid/rec/bulgarian': 0.7895791583166333, 'epoch': 8}
{'valid/acc/croatian': 0.388671875, 'valid/f1/croatian': 0.19948849104859334, 'valid/prec/croatian': 0.11504424778761062, 'valid/rec/croatian': 0.75, 'epoch': 8}
{'valid/acc/estonian': 0.4016393442622951, 'valid/f1/estonian': 0.16571428571428573, 'valid/prec/estonian': 0.09634551495016612, 'valid/rec/estonian': 0.5918367346938775, 'epoch': 8}
{'valid/acc/hungarian': 0.474025974025974, 'valid/f1/hungarian': 0.6292906178489702, 'valid/prec/hungarian': 0.4833040421792619, 'valid/rec/hungarian': 0.9016393442622951, 'epoch': 8}


0it [00:00, ?it/s]

{'valid/acc/russian': 0.463, 'valid/f1/russian': 0.48414985590778103, 'valid/prec/russian': 0.37555886736214605, 'valid/rec/russian': 0.6810810810810811, 'epoch': 8}


304it [01:45,  2.89it/s]


{'valid/acc/english': 0.6265457543281121, 'valid/f1/english': 0.6384676775738228, 'valid/prec/english': 0.6201550387596899, 'valid/rec/english': 0.6578947368421053, 'epoch': 9}
{'valid/acc/bulgarian': 0.3769161406672678, 'valid/f1/bulgarian': 0.35840297121634174, 'valid/prec/bulgarian': 0.23323262839879155, 'valid/rec/bulgarian': 0.7735470941883767, 'epoch': 9}
{'valid/acc/croatian': 0.390625, 'valid/f1/croatian': 0.1958762886597938, 'valid/prec/croatian': 0.1130952380952381, 'valid/rec/croatian': 0.7307692307692307, 'epoch': 9}
{'valid/acc/estonian': 0.4385245901639344, 'valid/f1/estonian': 0.15950920245398775, 'valid/prec/estonian': 0.09386281588447654, 'valid/rec/estonian': 0.5306122448979592, 'epoch': 9}
{'valid/acc/hungarian': 0.46915584415584416, 'valid/f1/hungarian': 0.6245694603903559, 'valid/prec/hungarian': 0.48056537102473496, 'valid/rec/hungarian': 0.8918032786885246, 'epoch': 9}
{'valid/acc/russian': 0.47, 'valid/f1/russian': 0.47937131630648333, 'valid/prec/russian': 0.37